In [10]:
from PIL import Image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Dropout, GaussianNoise, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from tensorflow.keras.callbacks import EarlyStopping
from keras.losses import binary_crossentropy
from tensorflow.keras.regularizers import l2
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

In [2]:
height, width, channels = 372, 1244, 1  #input image format
input_img = Input(shape=(height, width, channels))
#print(input_img)

In [11]:
x = GaussianNoise(0.1)(input_img)
# Encoder
x = Conv2D(64, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(0.001))(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Dropout(0.3)(x)  # Adjusted dropout rate
x = Conv2D(32, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(0.001))(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Dropout(0.3)(x)

# Decoder
x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = UpSampling2D((2, 2))(x)
x = Dropout(0.3)(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = UpSampling2D((2, 2))(x)
x = Dropout(0.3)(x)
decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

In [12]:
autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
autoencoder.summary()

def preprocess_image(image):
    #resize image so they're same format
    image = tf.image.resize(image, (height, width), method='lanczos5')
    return image


datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,  # Rotates the images by up to 10 degrees
    width_shift_range=0.1,  # Shifts the image width by a maximum of 10%
    height_shift_range=0.1,  # Shifts the image height by a maximum of 10%
    shear_range=0.1,  # Shear angle in counter-clockwise direction
    zoom_range=0.1,  # Random zoom
    horizontal_flip=True,  # Randomly flip inputs horizontally
    fill_mode='nearest',  # Strategy to fill newly created pixels
    preprocessing_function=preprocess_image,
    validation_split=0.2
)


Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 372, 1244, 1)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gaussian_noise_3                │ (None, 372, 1244, 1)   │             0 │
│ (GaussianNoise)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 372, 1244, 64)  │           640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 372, 1244, 64)  │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 186, 622, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 186, 622, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_12 (Conv2D)              │ (None, 186, 622, 32)   │        18,464 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 186, 622, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 93, 311, 32)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_9 (Dropout)             │ (None, 93, 311, 32)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_13 (Conv2D)              │ (None, 93, 311, 32)    │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 93, 311, 32)    │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d_4 (UpSampling2D)  │ (None, 186, 622, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_10 (Dropout)            │ (None, 186, 622, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_14 (Conv2D)              │ (None, 186, 622, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 186, 622, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d_5 (UpSampling2D)  │ (None, 372, 1244, 64)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_11 (Dropout)            │ (None, 372, 1244, 64)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_15 (Conv2D)              │ (None, 372, 1244, 1)   │           577 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 48,193 (188.25 KB)

 Trainable params: 47,809 (186.75 KB)

 Non-trainable params: 384 (1.50 KB)

In [5]:
import os
os.getcwd()

'/Users/yong/Desktop'

In [13]:
train_generator = datagen.flow_from_directory(
    'thesis/dataset',  #path to dataset
    target_size=(height, width),
    color_mode='grayscale',
    batch_size=16,
    class_mode='input',
    subset='training'
)
validation_generator = datagen.flow_from_directory(
    'thesis/dataset',  #path to dataset
    target_size=(height, width),
    color_mode='grayscale',
    batch_size=16,
    class_mode='input',
    subset='validation'
)

Found 69 images belonging to 5 classes.


Found 15 images belonging to 5 classes.


In [14]:
# Define early stopping callback
early_stopper = EarlyStopping(monitor='val_loss', patience=5, verbose=1, restore_best_weights=True)

# Now include the callback in the fit method
autoencoder.fit(
    train_generator,
    epochs=40,
    validation_data=validation_generator,
    callbacks=[early_stopper]
)

Epoch 1/40


/Users/yong/anaconda3/envs/plard/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


5/5 ━━━━━━━━━━━━━━━━━━━━ 192s 37s/step - loss: 0.2465 - val_loss: 0.2325
Epoch 2/40
5/5 ━━━━━━━━━━━━━━━━━━━━ 154s 37s/step - loss: 0.0825 - val_loss: 0.1608
Epoch 3/40
5/5 ━━━━━━━━━━━━━━━━━━━━ 179s 43s/step - loss: 0.0589 - val_loss: 0.0973
Epoch 4/40
5/5 ━━━━━━━━━━━━━━━━━━━━ 166s 31s/step - loss: 0.0526 - val_loss: 0.0687
Epoch 5/40
5/5 ━━━━━━━━━━━━━━━━━━━━ 216s 34s/step - loss: 0.0467 - val_loss: 0.0547
Epoch 6/40
5/5 ━━━━━━━━━━━━━━━━━━━━ 182s 36s/step - loss: 0.0448 - val_loss: 0.0518
Epoch 7/40
5/5 ━━━━━━━━━━━━━━━━━━━━ 152s 36s/step - loss: 0.0413 - val_loss: 0.0845
Epoch 8/40
5/5 ━━━━━━━━━━━━━━━━━━━━ 159s 31s/step - loss: 0.0390 - val_loss: 0.1069
Epoch 9/40
5/5 ━━━━━━━━━━━━━━━━━━━━ 163s 30s/step - loss: 0.0369 - val_loss: 0.1066
Epoch 10/40
5/5 ━━━━━━━━━━━━━━━━━━━━ 175s 35s/step - loss: 0.0336 - val_loss: 0.1194
Epoch 11/40
5/5 ━━━━━━━━━━━━━━━━━━━━ 179s 35s/step - loss: 0.0315 - val_loss: 0.1350
Epoch 11: early stopping
Restoring model weights from the end of the best epoch: 6.


In [15]:
#save model
autoencoder.save('0504_4.keras')